In [7]:
import os
import subprocess
import re
import pandas as pd

encoding="cp932"
def searchFile(ROOT_PATH,saveFile):
    if not os.path.isdir(os.path.dirname(saveFile)):
        os.makedirs(os.path.dirname(saveFile))
    allFile={}
    count=0
    for pathname,dirnames,filenames in os.walk(ROOT_PATH):
        for filename in filenames:
            if filename.endswith(".java"):
                fullPath=os.path.join(pathname,filename)
                count=count+1
                if "info" not in fullPath:
                    parentFolder=re.sub(r"\\src\\.*","",fullPath)
                    if parentFolder in allFile:
                        allFile[parentFolder].append(fullPath)
                    else:
                        allFile.setdefault(parentFolder,[fullPath])
                        
    with open(saveFile,"w",encoding="utf-8") as f:              
        for key,item in allFile.items():
            for pathfile in item:
                print(pathfile,file=f)
            print(file=f)
    
def editClass(Spoonfile, CKfile, outPutFile):   
    df = pd.read_csv(CKfile,encoding=encoding)
    df = df[["file", "class","type", "dit", "fanin", "fanout","lcc","lcom*","loc","noc","rfc","innerClassesQty","totalFieldsQty","totalMethodsQty","wmc"]]
    CK=df.copy()
    CK["class"] = CK["class"].str.replace("$Anonymous", "$")
    spoon = pd.read_csv(Spoonfile,encoding=encoding)
    join = spoon.merge(CK, how="inner", on=["file", "class"])
    join["class"]=join["class"].str.replace("$",".",)
    join.to_csv(
        outPutFile,mode="w",index=False,encoding=encoding
    )

def editMethod(SpoonFile, CKFile, outPutFile):
    df = pd.read_csv(CKFile,encoding=encoding)
    df = df[["file", "class", "method","fanin","fanout", "wmc", "loc","returnsQty","variablesQty","parametersQty","loopQty","anonymousClassesQty","innerClassesQty"]]
    CK=df.copy()
    CK["class"] = CK["class"].str.replace("$Anonymous", "$")    
    spoon = pd.read_csv(SpoonFile,encoding=encoding)
    join = spoon.merge(CK, how="inner", on=["file", "class", "method"])
    join["class"]=join["class"].str.replace("$",".")
    join.to_csv(
        outPutFile,mode="w",index=False,encoding=encoding
    )
  
def editFile(saveMetricsFile,outPutFile):
    CKClass=saveMetricsFile+"class.csv"
    CKMethod=saveMetricsFile+"method.csv"
    SpoonClass=saveMetricsFile+"spoonClass.csv"
    SpoonMethod=saveMetricsFile+"spoonMethod.csv"
    saveClass=outPutFile+"joinClass.csv"
    saveMethod=outPutFile+"joinMethod.csv"
    
    editClass(SpoonClass,CKClass,saveClass)
    editMethod(SpoonMethod,CKMethod,saveMethod)
      
searchFile(os.path.join("C:\\Users\\syuuj","gitProject\\jsoup"),os.path.join("C:\\Users\\syuuj","getData_python\\full.txt"))

In [ ]:
import pandas as pd
import shutil

def getCodeSmellClass():
    dfGod=pd.read_csv("ground truth\\GodClass.csv")
    dfRefused=pd.read_csv("ground truth\\RefusedBequest.csv")
    dfGod=dfGod[["nomeSistema","class"]]
    dfRefused=dfRefused[["nomeSistema","class"]]
    allClass=pd.concat([dfGod,dfRefused])
    allClass=allClass.drop_duplicates()
    return allClass
   
def createFolder(path):
    if os.path.isdir(path):
        shutil.rmtree(path)
    os.makedirs(path)

RootFolder="C:\\Users\\syuuj"
with open("tmp.txt","r",encoding="utf-8") as f:
    string=f.read() 
Projects=string.split("\n")

for block in Projects:
    blockFactor=block.split(",")
    smellFileName=blockFactor[3]
    project=os.path.join(RootFolder,"gitProject",blockFactor[0])      
    checkoutVersions=[blockFactor[1],blockFactor[2]]                                        

    saveFolder=os.path.join(os.getcwd(),"data",os.path.basename(project))
    saveTextFile=os.path.join(saveFolder,"full.txt")
    CKAnalyze=["java","-jar","ck-0.7.1-SNAPSHOT-jar-with-dependencies.jar",project,"False","0","True",saveFolder]
    SpoonAnalyze=["java","-jar","demo-1.0-snapshot.jar",saveTextFile,saveFolder]

    createFolder(saveFolder)

    allClass=getCodeSmellClass()
    group=allClass.groupby("nomeSistema")
    classes=group.get_group(smellFileName)

    i=1
    for version in checkoutVersions:
        gitCheckout=["git","checkout",version]
        subprocess.run(gitCheckout,cwd=project,check=True)
        searchFile(project,saveTextFile)
        SaveFile=os.path.join(saveFolder,str(i))
        CKAnalyze[7]=SaveFile
        SpoonAnalyze[4]=SaveFile
        subprocess.run(CKAnalyze,cwd=os.path.join(RootFolder,"ck\\target"),check=True)
        subprocess.run(SpoonAnalyze,cwd=os.path.join(os.getcwd(),"spoon\\target"),check=True)
        outPutFile=os.path.join(saveFolder,str(i))
        editFile(SaveFile,outPutFile)
        if i==1:
            print(smellFileName)
            df=pd.read_csv(outPutFile+"joinClass.csv",encoding=encoding)
            print(len(set(df['class']) & set(classes['class'])),"/",len(classes))
            data=(set(df['class']) & set(classes['class']))
            print(classes[~classes['class'].isin(df['class'])])

        i=i+1

fastjson
31 / 31
Empty DataFrame
Columns: [nomeSistema, class]
Index: []


問題点：クラス数，メソッド数が微妙に違う

        ファイル名が異なるものがある

        ツールで計算する時，エラー・終了しない(checkstyle,jitwatch)

        ファイル名が長く保存できない(shenyu)

In [4]:
code=['java', '-jar', 'demo-1.0-snapshot.jar', 'c:\\Users\\syuuj\\getData_python\\data\\dbeaver\\full.txt', 'c:\\Users\\syuuj\\getData_python\\data\\dbeaver\\1']
print(" ".join(code))

java -jar demo-1.0-snapshot.jar c:\Users\syuuj\getData_python\data\dbeaver\full.txt c:\Users\syuuj\getData_python\data\dbeaver\1


In [5]:
import pandas as pd
import shutil

def getCodeSmellClass():
    dfGod=pd.read_csv("ground truth\\GodClass.csv")
    dfRefused=pd.read_csv("ground truth\\RefusedBequest.csv")
    dfGod=dfGod[["nomeSistema","class"]]
    dfRefused=dfRefused[["nomeSistema","class"]]
    allClass=pd.concat([dfGod,dfRefused])
    allClass=allClass.drop_duplicates()
    return allClass
   
def createFolder(path):
    if os.path.isdir(path):
        shutil.rmtree(path)
    os.makedirs(path)

def doAmalyze():
    with open("tmp.txt","r",encoding="utf-8")as f:
        tmpString=f.read()
    Blocks=tmpString.split("\n")
    for Block in Blocks:
        ListFactor=Block.split(",")
        RootFolder="C:\\Users\\syuuj"
        smellFileName=ListFactor[2]
        project=os.path.join(RootFolder,"gitProject",ListFactor[0])      
        version=ListFactor[1]
        saveFolder=os.path.join(os.getcwd(),os.path.basename(project))
        saveTextFile=os.path.join(saveFolder,"full.txt")
        CKAnalyze=["java","-jar","ck-0.7.1-SNAPSHOT-jar-with-dependencies.jar",project,"False","0","True",saveFolder]
        SpoonAnalyze=["java","-jar","demo-1.0-snapshot.jar",saveTextFile,saveFolder]

        createFolder(saveFolder)

        allClass=getCodeSmellClass()
        group=allClass.groupby("nomeSistema")
        classes=group.get_group(smellFileName)

        i=1
        gitCheckout=["git","checkout",version]
        subprocess.run(gitCheckout,cwd=project,check=True)
        searchFile(project,saveTextFile)
        SaveFile=os.path.join(saveFolder,str(i))
        CKAnalyze[7]=SaveFile
        SpoonAnalyze[4]=SaveFile
        subprocess.run(CKAnalyze,cwd=os.path.join(RootFolder,"ck\\target"),check=True)
        subprocess.run(SpoonAnalyze,cwd=os.path.join(os.getcwd(),"spoon\\target"),check=True)
        outPutFile=os.path.join(saveFolder,str(i))
        editFile(SaveFile,outPutFile)
        if i==1:
            print(smellFileName)
            df=pd.read_csv(outPutFile+"joinClass.csv",encoding=encoding)
            print(len(set(df['class']) & set(classes['class'])),"/",len(classes))
            data=(set(df['class']) & set(classes['class']))
            print(classes[~classes['class'].isin(df['class'])])
        i=i+1
    
doAmalyze()

dbeaver
116 / 116
Empty DataFrame
Columns: [nomeSistema, class]
Index: []


IndexError: list index out of range